In [ ]:
from langchain import OpenAI, ConversationChain
from langchain.schema import BaseMemory
from pydantic import BaseModel
from typing import List, Dict, Any

In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_lg

In [ ]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [ ]:
from langchain import OpenAI, ConversationChain
from langchain.schema import BaseMemory
from pydantic import BaseModel
from typing import List, Dict, Any
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['novella']
storyCollection = db['story']
class NovellaStoryMemory(BaseMemory, BaseModel):
    """Memory class for storing information about entities."""

    # Define dictionary to store information about entities.
    chapters: dict = {}
    # Define key to pass information about chapters into prompt.
    memory_key: str = "story info"

    def clear(self):
        self.chapters = {}

    @property
    def memory_variables(self) -> List[str]:
        """Define the variables we are providing to the prompt."""
        return [self.memory_key]

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Load the memory variables, in this case the entity key."""
        # Get the story input
        doc = storyCollection.find_one({})['chapters']
        # Extract known information about chapters, if they exist.
        chapters = doc
        # Return combined information about chapters to put into context.
        return {self.memory_key: "\n".join(chapters)}

In [ ]:
from langchain.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. You are provided with information about entities the Human mentions, if relevant.

Relevant entity information:
{entities}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["entities", "input"], template=template)

In [ ]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, prompt=prompt, verbose=True, memory=SpacyEntityMemory()
)

In [ ]:
conversation.predict(input="Harrison likes machine learning")



In [ ]:
conversation.predict(
    input="What do you think Harrison's favorite subject in college was?"
)